In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2yoq3c70
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2yoq3c70
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=885a9992e44c0714ce12470db397356f88331e9f3a5fe0aa3e549ec83fc4d92e
  Stored in directory: /tmp/pip-ephem-wheel-cache-yuqwr3ia/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


**Load the Dataset and unzip it**

In [3]:
!rm -r *
!wget -q https://www.dropbox.com/s/ddxg7yfww8mbgxf/Lab8.zip
!unzip Lab8.zip
!rm Lab8.zip
!ls

Archive:  Lab8.zip
   creating: Dataset/Test/
   creating: Dataset/Test/0/
  inflating: Dataset/Test/0/input.raw  
  inflating: Dataset/Test/0/myOutput.raw  
  inflating: Dataset/Test/0/output.raw  
   creating: Dataset/Test/1/
  inflating: Dataset/Test/1/input.raw  
  inflating: Dataset/Test/1/myOutput.raw  
  inflating: Dataset/Test/1/output.raw  
   creating: Dataset/Test/10/
  inflating: Dataset/Test/10/input.raw  
  inflating: Dataset/Test/10/myOutput.raw  
  inflating: Dataset/Test/10/output.raw  
   creating: Dataset/Test/11/
  inflating: Dataset/Test/11/input.raw  
  inflating: Dataset/Test/11/myOutput.raw  
  inflating: Dataset/Test/11/output.raw  
   creating: Dataset/Test/2/
  inflating: Dataset/Test/2/input.raw  
  inflating: Dataset/Test/2/myOutput.raw  
  inflating: Dataset/Test/2/output.raw  
   creating: Dataset/Test/3/
  inflating: Dataset/Test/3/input.raw  
  inflating: Dataset/Test/3/myOutput.raw  
  inflating: Dataset/Test/3/output.raw  
   creating: Dataset/Test/4/

# **CUDA 2D Scan Code**

Code to be modified where there are TODO indications

In [4]:
%%writefile lab8-SCAN.cu
// 2D Reduction program
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#define BLOCK_SIZE 512 //TODO: You can change this
#define TRANSPOSE_TILE_DIM 32
#define TRANSPOSE_BLOCK_ROWS 8

float* ImportMatrix(char* filename, int* N, int* M,int mode)
{
    FILE* handle;
	float* val;
    int i, j,tmp=0;
	float ftmp=0;
    printf("Reading File %s\n", filename);
    if (filename == NULL) {
        return NULL;
    }

    handle = fopen(filename, "r");
    if (handle == NULL) {
        printf("Error opening file: %s\n", filename);
        return NULL;
    }
    fscanf(handle, "%d %d", N, M);
    val = (float*)malloc(*N * *M * sizeof(float));
    for (i = 0; i < *N; i++) {
        for (j = 0; j < *M; j++) {
		   if(mode==0){
            fscanf(handle, "%d ", &tmp);
			val[(i * *M) + j]=(float)tmp;
		 }else{
		    fscanf(handle, "%f ", &ftmp);
			val[(i * *M) + j]=ftmp;

		 }

        }
    }
    fclose(handle);
    return val;
}
int ExportMatrix(char* filename, float *val, int N, int M)
{
	FILE* handle;
	int i,j;
	printf("Writing File %s\n", filename);
	if (filename == NULL) {
		return 0;
	}
	handle = fopen(filename, "w");

	if (handle == NULL) {
		printf("Error opening file: %s\n", filename);
		return 0;
	}

	fprintf(handle, "%d %d\n", N,M);
	for (i = 0; i < N; i++) {
	  for(j=0;j<M;j++){
		fprintf(handle, "%.1f ", val[i*M+j]);
	  }
	  fprintf(handle,"\n");
	}
	fclose(handle);
	return 1;
}


clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}
__global__ void fixup(float *input, float *aux, int len) {
	unsigned int t = threadIdx.x, start = 2 * blockIdx.x * BLOCK_SIZE;
	if (blockIdx.x) {
		if (start + t < len)
			input[start + t] += aux[blockIdx.x];
		if (start + BLOCK_SIZE + t < len)
			input[start + BLOCK_SIZE + t] += aux[blockIdx.x];
	}
}

__global__ void transposeCoalesced(float *odata, const float *idata, int numInputRows, int numInputCols)
{
	__shared__ float tile[TRANSPOSE_TILE_DIM][TRANSPOSE_TILE_DIM + 1];

	// TODO:Read data from global memory to shared meory

	int x = blockIdx.x * TRANSPOSE_TILE_DIM + threadIdx.x;
	int y = blockIdx.y * TRANSPOSE_TILE_DIM + threadIdx.y;

	for (int i = 0; i < TRANSPOSE_TILE_DIM; i+=TRANSPOSE_BLOCK_ROWS) {
		if (y + i < numInputRows && x < numInputCols) {
			tile[threadIdx.y + i][threadIdx.x] = idata[(y+i)*numInputCols+x];
		}

	}

	__syncthreads();
	// TODO:Tanspose data from using shared meory to odata

	x = blockIdx.y * TRANSPOSE_TILE_DIM + threadIdx.x;
  y = blockIdx.x * TRANSPOSE_TILE_DIM + threadIdx.y;


	for (int i = 0; i < TRANSPOSE_TILE_DIM; i+=TRANSPOSE_BLOCK_ROWS) {
		if (y + i < numInputCols && x < numInputRows)
			odata[(y+i)*numInputRows + x] = tile[threadIdx.x][threadIdx.y + i];
	}


}

__global__ void scan(float *input, float *output, float *aux, int len) {
	// Modify the body of this function to complete the functionality of
	// the scan on the device
	// You may need multiple kernel calls; write your kernels before this
	// function and call them from here

	// Load a segment of the input vector into shared memory
	__shared__ float scan_array[BLOCK_SIZE << 1];
	unsigned int thid = threadIdx.x, start = 2 * blockIdx.x * BLOCK_SIZE;

	int ai = thid;
	int bi = thid + BLOCK_SIZE;
	float sum;

	scan_array[ai] = (start + ai < len) ? input[start + ai] : 0;
	scan_array[bi] = (start + bi < len) ? input[start + bi] : 0;

	int offset = 1;

	// TODO: build the sum in place up the tree


	for (unsigned int stride = BLOCK_SIZE; stride > 0; stride /= 2) {
		 __syncthreads();

		 if (thid < stride) {
			 ai = offset * (2 * thid+1)-1;
			 bi = offset * (2 * thid+2)-1;

			 scan_array[bi] += scan_array[ai];
		 }

		 offset *= 2;
	}

	if (thid == 0) {
		sum = scan_array[2*BLOCK_SIZE -1];
		scan_array[2*BLOCK_SIZE-1] = 0;
	}

	// TODO: traverse down the tree building the scan in place
	for (int stride = 1; stride < 2 * BLOCK_SIZE; stride *= 2) {
    __syncthreads();
		offset >>= 1;

		if (thid < stride) {
			 ai = offset * (2 * thid+1)-1;
			 bi = offset * (2 * thid+2)-1;

			 float temp = scan_array[ai];
			 scan_array[ai] = scan_array[bi];
			 scan_array[bi] += temp;
		}
  }

	__syncthreads();
	if (start+ai < len ) {
		output[start + ai] = scan_array[ai];
	}

	if (start + bi < len) {
		output[start + bi] = scan_array[bi];
	}

	if (aux && thid == 0) {
		aux[blockIdx.x] = sum;
	}
}

void rscan(float* deviceInput, float *deviceOutput, int numElements)
{
	float *deviceAuxArray, *deviceAuxScannedArray;

	int numBlocks = ceil((float)numElements / (BLOCK_SIZE << 1));
	cudaMalloc(&deviceAuxArray, numBlocks * sizeof(float));

	scan << <numBlocks, BLOCK_SIZE >> >(deviceInput, deviceOutput, deviceAuxArray,numElements);
	cudaPeekAtLastError();
	cudaDeviceSynchronize();

	if (numBlocks > 1) {
		cudaMalloc(&deviceAuxScannedArray, numBlocks * sizeof(float));

		rscan(deviceAuxArray, deviceAuxScannedArray, numBlocks);

		fixup << <numBlocks, BLOCK_SIZE >> >(deviceOutput, deviceAuxScannedArray,numElements);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

		cudaFree(deviceAuxScannedArray);
	}

	cudaFree(deviceAuxArray);
}


int main(int argc, char **argv) {
	int  i;
	float *hostInput;  // The input 1D list
	float *hostOutput; // The output list
	float *hostReference; //The reference solution
	float *deviceInput;
	float *deviceTmpOutput;
	float *deviceOutput;
	int numInputRows, numInputCols; // number of elements in the list
	int numInputRowsR, numInputColsR; // number of elements in the list
	int prt = 0;
	float meanDiff = 0;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[3];

for (ti=0;ti<12;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.raw",ti);
  sprintf(argv2,"Dataset/Test/%i/input.raw",ti);
	sprintf(argv3,"Dataset/Test/%i/Myoutput.raw",ti);
	sprintf(argv4,"ON");

	printf("Running GPU Reduction Using Simple Algorithm...\n");
	if (strcmp(argv4, "ON") == 0)prt = 1;

  StartingTime = Time_start();
	hostInput =		(float*)ImportMatrix(argv2, &numInputRows,&numInputCols,0);
	printf("Matrix Size %d %d\n",numInputRows,numInputCols);
	hostReference = (float*)ImportMatrix(argv1, &numInputRowsR,&numInputColsR,1);
	hostOutput = (float *)malloc(numInputRows *numInputCols * sizeof(float));
	Elapsed_time(StartingTime, "Importing data and creating memory on host", prt);

  Elapsed_time(StartingTime, "Reading Data.", prt);

	/*TODO: Allocate GPU Memory*/

	StartingTime=Time_start();

	cudaMalloc((void **)&deviceInput, numInputRows *numInputCols  * sizeof(float));
	cudaMalloc((void **)&deviceOutput, numInputRows *numInputCols  * sizeof(float));
	cudaMalloc((void **)&deviceTmpOutput, numInputRows * numInputCols * sizeof(float));


	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
	cudaMemcpy(deviceInput, hostInput, numInputRows *numInputCols  * sizeof(float), cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");
	//TODO: Modify this to complete the functionality of the scan on the deivce
	StartingTime = Time_start();
	for (i = 0; i < numInputRows; ++i) {
		rscan(deviceInput + i * numInputCols, deviceTmpOutput + i * numInputCols, numInputCols);
		cudaDeviceSynchronize();
	}
	Elapsed_time(StartingTime, "Perform rscan on the GPU", prt);

	// TODO: Perform kernel computation here
	StartingTime = Time_start();

	dim3 transposeBlockDim(TRANSPOSE_TILE_DIM, TRANSPOSE_BLOCK_ROWS);
	dim3 transposeGridDim(ceil(numInputCols / (float)TRANSPOSE_TILE_DIM), ceil(numInputRows / (float)TRANSPOSE_TILE_DIM));

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
	transposeCoalesced << <transposeGridDim, transposeBlockDim >> >(deviceOutput, deviceTmpOutput, numInputRows, numInputCols);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Tranposed Coaleced Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	StartingTime = Time_start();

	for (i = 0; i < numInputCols; ++i) {
		rscan(deviceOutput + i * numInputRows, deviceTmpOutput + i * numInputRows, numInputRows);
		cudaDeviceSynchronize();
	}
	Elapsed_time(StartingTime, "-------------->Rescan ***************", prt);

	transposeBlockDim = dim3(TRANSPOSE_TILE_DIM, TRANSPOSE_BLOCK_ROWS);
	transposeGridDim = dim3(ceil(numInputRows / (float)TRANSPOSE_TILE_DIM), ceil(numInputCols / (float)TRANSPOSE_TILE_DIM));
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
	transposeCoalesced << <transposeGridDim, transposeBlockDim >> >(deviceOutput, deviceTmpOutput, numInputCols, numInputRows);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Tranposed Coaleced Elapsed Time -II*********** %f in mili-seconds\n", gpu_time);



	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
	cudaMemcpy(hostOutput, deviceOutput, numInputRows *numInputCols * sizeof(float), cudaMemcpyDeviceToHost);

	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
	cudaFree(deviceInput);
	cudaFree(deviceOutput);
	cudaFree(deviceTmpOutput);

	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);


/*Perform Success Test*/
meanDiff=0.0;
	for (i = 0; i < numInputRows *numInputCols; i++) {
		meanDiff = meanDiff + fabs(hostOutput[i] - hostReference[i]);
	}
	meanDiff = meanDiff / (float)(numInputRows *numInputCols);
	if (meanDiff >0.0001) {
		printf("%f Failed\n",meanDiff);

	}
	else {
		printf("%f Passed\n",meanDiff);
	}
	ExportMatrix(argv3, hostOutput, numInputRows,numInputCols);

	free(hostInput);
	free(hostOutput);
	free(hostReference);


}

	return 1;
}


Writing lab8-SCAN.cu


**Compile GPU Scan Code**

In [5]:
!nvcc  lab8-SCAN.cu -o lab8-SCAN.exe
!ls

Dataset  lab8-SCAN.cu  lab8-SCAN.exe


**Execute GPU Scan Code**

In [6]:
!./lab8-SCAN.exe



 +++++++++++++Test 0
Running GPU Reduction Using Simple Algorithm...
Reading File Dataset/Test/0/input.raw
Matrix Size 16 16
Reading File Dataset/Test/0/output.raw
Importing data and creating memory on host Elapsed Time 0.202000 in mili-seconds
Reading Data. Elapsed Time 0.213000 in mili-seconds
Allocate GPU Memory Elapsed Time 203.075000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.116000 in mili-seconds
Launching kernel
Perform rscan on the GPU Elapsed Time 0.432000 in mili-seconds
-------->Tranposed Coaleced Elapsed Time*********** 0.010592 in mili-seconds
-------------->Rescan *************** Elapsed Time 0.370000 in mili-seconds
-------->Tranposed Coaleced Elapsed Time -II*********** 0.009728 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.028000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.176000 in mili-seconds
0.000000 Passed
Writing File Dataset/Test/0/Myoutput.raw


 +++++++++++++Test 1
Running GPU Reduction Using Simple Algorithm..

**Save trace into Lab8SCAN.txt**